In [1]:
import numpy as np;
from scipy.optimize import minimize 
import time
import matplotlib.pyplot as plt
import sklearn.linear_model
import cv2


In [2]:
def aul_f(X, Y, Z, n, m, u, mu, tensor, entries = None):
    nx, ny, nz = n
    val = 0;
    val += np.sum(X**2);
    for i in range(m):
        val += np.sum(Y[i]**2) * np.sum(Z[i]**2);
    
    if entries is None:
        ten = np.zeros(nx*ny*nz);
        for i in range(m):
            ten += np.kron(X[i], np.kron(Y[i], Z[i]))
        val = val - np.dot(u.T, (ten - tensor));
        val = val+np.sum((ten - tensor)**2)*(1/(2*mu));
    else:
        for xi in entries.keys():
            for yi in entries[xi].keys():
                for zi in entries[xi][yi]:
                    ent = -tensor[xi*ny*nz+nz*yi+zi]
                    for j in range(m):
                        ent += X[j, xi]*Y[j, yi]*Z[j, zi]
                    val += (1/(2*mu))* ent**2;
                    
    return val;

In [3]:
def grad_aulf(X, n, m, u, mu, tensor, entries = None):
    grad = np.zeros(X.shape);
    grad[:n*m] = X[:n*m]*2.0;
    for i in range(m):
        grad[n*m+i*n:n*m+(i+1)*n] = 2*X[n*m+i*n:n*m+(i+1)*n]*np.sum(X[2*n*m+i*n:2*n*m+(i+1)*n]**2);
        grad[2*n*m+i*n:2*n*m+(i+1)*n] = 2*X[2*n*m+i*n:2*n*m+(i+1)*n]*np.sum(X[n*m+i*n:n*m+(i+1)*n]**2);
    ten = np.zeros(n*n*n);
    for i in range(m):
        ten += np.kron(X[i*n: (i+1)*n], np.kron(X[n*m+i*n: n*m+(i+1)*n], X[2*n*m+i*n: 2*n*m+(i+1)*n]))
    ten = ten - tensor;
    ten = (1/mu)*ten - u;
    ten1 = ten.reshape(n, n*n)
    ten2 = ten1.T.reshape(n, n*n)
    ten3 = ten2.T.reshape(n, n*n)
    for i in range(m):
        grad[i*n:(i+1)*n] += np.dot(ten1, 
                                     np.kron(X[n*m+i*n: n*m+(i+1)*n], X[2*n*m+i*n: 2*n*m+(i+1)*n]))
        grad[n*m+i*n:n*m+(i+1)*n] += np.dot(ten2, 
                                         np.kron(X[2*n*m+i*n: 2*n*m+(i+1)*n], X[i*n: (i+1)*n]))
        grad[2*n*m+i*n:2*n*m+(i+1)*n] += np.dot(ten3, 
                                                 np.kron(X[i*n: (i+1)*n], X[n*m+i*n: n*m+(i+1)*n]))
    return grad;



def compute_tensor(X, Y, Z, n, m):
    nx, ny, nz = n
    ten = np.zeros(nx*ny*nz);
    for i in range(m):
        ten += np.kron(X[i], np.kron(Y[i], Z[i]))
    return ten;

def compute_nuc_approx(X, Y, Z, m):
    val = 0;
    for i in range(m):
        val+= np.linalg.norm(X[i])*np.linalg.norm(Y[i])*np.linalg.norm(Z[i])
    return val

In [17]:
def reg_update_x(X, Y, Z, n, m, u, mu, entries_idx, entries, num_entries):
    nx, ny, nz = n
    num_sampl = num_entries+nx*m
    num_feat = nx*m
    M = np.zeros((num_sampl, num_feat))
    B = np.zeros(num_sampl)
    W = np.zeros(num_sampl)+1/(2*mu)
    for i in range(nx*m):
        M[i, i] = 1
        W[i] = 1;
    row = nx*m;
    for t in range(num_entries):
        xi = entries_idx[t, 0]
        yi = entries_idx[t, 1]
        zi = entries_idx[t, 2]
        for j in range(m):
            M[row, j*nx+xi]+= Y[j, yi]*Z[j, zi]
        B[row] = entries[t]
        row += 1
    opt = sklearn.linear_model.LinearRegression(fit_intercept=False)
    opt.fit(M, B, W)
    X = opt.coef_.reshape((m, nx))
    return X

def reg_update_y(X, Y, Z, n, m, u, mu, entries_idx, entries, num_entries):
    nx, ny, nz = n
    num_sampl = num_entries+ny*m
    num_feat = ny*m
    M = np.zeros((num_sampl, num_feat))
    B = np.zeros(num_sampl)
    W = np.zeros(num_sampl)+1/(2*mu)
    for i in range(ny*m):
        M[i, i] = 1
        W[i] = np.sum(Z[i//ny]**2)
    row = ny*m;
    for t in range(num_entries):
        xi = entries_idx[t, 0]
        yi = entries_idx[t, 1]
        zi = entries_idx[t, 2]
        for j in range(m):
            M[row, j*ny+yi]+=X[j, xi]*Z[j, zi]
        B[row] = entries[t]
        row += 1
    opt = sklearn.linear_model.LinearRegression(fit_intercept=False)
    opt.fit(M, B, W)
    Y = opt.coef_.reshape((m, ny))
    return Y

def reg_update_z(X, Y, Z, n, m, u, mu, entries_idx, entries, num_entries):
    nx, ny, nz = n
    num_sampl = num_entries+nz*m
    num_feat = nz*m
    M = np.zeros((num_sampl, num_feat))
    B = np.zeros(num_sampl)
    W = np.zeros(num_sampl)+1/(2*mu)
    for i in range(nz*m):
        M[i, i] = 1
        W[i] = np.sum(Y[i//nz]**2)
    row = nz*m;
    for t in range(num_entries):
        xi = entries_idx[t, 0]
        yi = entries_idx[t, 1]
        zi = entries_idx[t, 2]
        for j in range(m):
            M[row, j*nz+zi] += Y[j, yi]*X[j, xi]
        B[row] = entries[t]
        row += 1
    opt = sklearn.linear_model.LinearRegression(fit_intercept=False)
    opt.fit(M, B, W)
    Z = opt.coef_.reshape((m, nz))
    return Z

In [5]:
def fix_components(X, Y, Z, n, m):
    nx, ny, nz = n
    for i in range(m):
        norm_x = np.sqrt(np.sqrt(np.sum(X[i]**2)))
        norm_yz = np.sqrt(np.sqrt(np.sum(Y[i]**2)*np.sum(Z[i]**2)))
        X[i] = X[i]*(norm_yz/norm_x)
        Y[i] = Y[i]*np.sqrt(norm_x/norm_yz)
        Z[i] = Z[i]*np.sqrt(norm_x/norm_yz)
    return (X, Y, Z)

In [22]:
def generate_ten_entries2(tensor, n, num, seed = None):
    nx, ny, nz = n;
    step = 0;
    if seed is not None:
        np.random.seed(seed)
    entries = np.zeros((num, 3), dtype = 'int');
    entries_val = np.zeros(num);
    entries_xyz = {}
    while (step<num):
        i = np.random.randint(nx);
        j = np.random.randint(ny);
        k = np.random.randint(nz);
        if (i not in entries_xyz.keys()):
            entries_xyz[i] = {}
        if (j not in entries_xyz[i].keys()):
            entries_xyz[i][j] = {}
        if (k not in entries_xyz[i][j].keys()):
            val = tensor[i,j,k];
            entries_xyz[i][j][k] = val;
            entries[step, 0] = i
            entries[step, 1] = j
            entries[step, 2] = k
            entries_val[step] = val
            step+=1;
    return entries, entries_val, entries_xyz

In [23]:
def eval_error_direct(X, Y, Z, n, m, tensor, num_trials = 1000):
    nx, ny, nz = n
    total_error = 0
    total_norm = 0
    for i in range(num_trials):
        x = np.random.randint(nx)
        y = np.random.randint(ny)
        z = np.random.randint(nz)
        prediction = 0
        for j in range(m):
            prediction += X[j, x] * Y[j, y] * Z[j, z]
        true_val = tensor[x, y, z]
        total_norm += np.square(true_val)
        total_error += np.square(prediction - true_val)
    return np.sqrt(total_error/total_norm)

In [26]:
n = (150, 150, 150);
nx, ny, nz = n
m = 7;
m1 = 9;
num_entries = 20000
np.random.seed(2021)
X_true = np.random.rand(nx*m).reshape((m, nx));
Y_true = np.random.rand(ny*m).reshape((m, ny));
Z_true = np.random.rand(nz*m).reshape((m, nz));
#Cor = np.random.rand(9*m*m).reshape((3*m, 3*m))
#Cor = Cor/(100*np.linalg.norm(Cor))
#Cor = Cor+np.identity(3*m);
#X_true = np.dot(X_true.reshape(n, 3*m), Cor).reshape((3*n*m,))
X_0 = np.random.rand(nx*m1).reshape((m1, nx));
Y_0 = np.random.rand(ny*m1).reshape((m1, ny));
Z_0 = np.random.rand(nz*m1).reshape((m1, nz));
u = np.zeros(nx*ny*nz);
mu = 1
tensor = compute_tensor(X_true, Y_true, Z_true, n, m)
a = compute_nuc_approx(X_true, Y_true, Z_true, m)
print(a)

entries_idx, entries, entries_xyz = generate_ten_entries2(tensor.reshape(n), n, num_entries, seed = 2021)

2498.7578043994445


In [27]:
prog_1 = np.zeros(500)
start = time.time()
for mu in [0.1, 0.001, 0.0001, 0.00001, 0.000001]:
    for step in range(20):
        X_0 = reg_update_x(X_0, Y_0, Z_0, n, m1, u, mu, entries_idx, entries, num_entries)
        Y_0 = reg_update_y(X_0, Y_0, Z_0, n, m1, u, mu, entries_idx, entries, num_entries)
        Z_0 = reg_update_z(X_0, Y_0, Z_0, n, m1, u, mu, entries_idx, entries, num_entries)
        X_0, Y_0, Z_0 = fix_components(X_0, Y_0, Z_0, n, m1)
        if (step%5 == 0):
            print(time.time() - start)
            print(aul_f(X_0, Y_0, Z_0, n, m1, u, mu, tensor, entries = entries_xyz))
            ten = compute_tensor(X_0, Y_0, Z_0, n, m1)
            prog_1[step//5] = np.sqrt(np.sum((ten - tensor)**2))
            print('F2 norm %f' % prog_1[step//5])
            err = eval_error_direct(X_0, Y_0, Z_0, n, m1, tensor.reshape(n))
            print('eval_error_direct %f' % err)
            print(time.time() - start)

15.988817930221558
8535.39277390658
F2 norm 442.638464
eval_error_direct 0.245251
16.501641035079956


KeyboardInterrupt: 